In [1]:
import sys
from matplotlib import rcParams 
rcParams["figure.figsize"] = (16,5)
sys.path.append('')

import librosa
import librosa.display

from TTS.tts.models.tacotron2 import Tacotron2 
from TTS.tts.utils import *
from TTS.tts.utils.generic_utils import setup_model
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config
from TTS.tts.utils.text import text_to_sequence, phoneme_to_sequence
from TTS.tts.utils.text.symbols import symbols, phonemes
import torch
from TTS.tts.utils.synthesis import synthesis
import numpy as np

In [2]:
# Set constants

# MODEL_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_rosana_only/checkpoint_120000.pth.tar'
CONFIG_PATH =  './experiments/debug_add_emb/config.json'

# MODEL_PATH ='/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/best_model.pth.tar'
# CONFIG_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/config.json'

CONFIG = load_config(CONFIG_PATH)
# CONFIG['datasets'][0]['path'] = './'
# CONFIG['output_path'] = './'
# CONFIG['audio']['signal_norm'] = False
# CONFIG['audio']['stats_path'] = ''
# CONFIG['use_phonemes'] = False
# CONFIG['save_step'] = 500
CONFIG['num_prosodic_features'] = 0
CONFIG['agg_style_space'] = False
CONFIG['lookup_speaker_dim'] = 512
CONFIG['use_style_lookup'] = False
CONFIG['lookup_style_dim'] = 64
CONFIG['use_prosodic_linear'] = False
CONFIG['prosodic_dim'] = 2
use_cuda = False

# load the model
ap = AudioProcessor(**CONFIG.audio)

num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)


# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, 3, 2, CONFIG, None)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:
 | > hop_length:256
 | > win_length:1024
 > Using model: Tacotron2
Training with 3 speakers and 2 styles
Use style target = False
Use semi supervised = False


In [3]:
print(model)

Tacotron2(
  (speaker_embedding): Embedding(3, 512)
  (embedding): Embedding(217, 512, padding_idx=0)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (activation): ReLU()
      )
      (1): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (activation): ReLU()
      )
      (2): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [4]:
model._add_speaker_embedding

<function TTS.tts.models.tacotron_abstract.TacotronAbstract._add_speaker_embedding(outputs, speaker_embeddings)>

In [5]:
import numpy as np

s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

def text_to_seqvec(text, CONFIG):
    text_cleaner = [CONFIG.text_cleaner]
    # text ot phonemes to sequence vector
    if CONFIG.use_phonemes:
        seq = np.asarray(
            phoneme_to_sequence(text, text_cleaner, CONFIG.phoneme_language,
                                CONFIG.enable_eos_bos_chars,
                                tp=CONFIG.characters if 'characters' in CONFIG.keys() else None),
            dtype=np.int32)
    else:
        seq = np.asarray(text_to_sequence(text, text_cleaner, tp=CONFIG.characters if 'characters' in CONFIG.keys() else None), dtype=np.int32)
    return seq


def numpy_to_torch(np_array, dtype, cuda=False):
    if np_array is None:
        return None
    tensor = torch.as_tensor(np_array, dtype=dtype)
    if cuda:
        return tensor.cuda()
    return tensor

def id_to_torch(speaker_id, cuda=False):
    if speaker_id is not None:
        speaker_id = np.asarray(speaker_id)
        speaker_id = torch.from_numpy(speaker_id).unsqueeze(0)
    if cuda:
        return speaker_id.cuda().type(torch.long)
    return speaker_id.type(torch.long)

def compute_style_mel(style_wav, ap, cuda=False):
    style_mel = torch.FloatTensor(ap.melspectrogram(
        ap.load_wav(style_wav, sr=ap.sample_rate))).unsqueeze(0)
    if cuda:
        return style_mel.cuda()
    return style_mel

In [26]:
speaker_id = id_to_torch(2, cuda=use_cuda)

inputs = text_to_seqvec(s, CONFIG)
inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
inputs = inputs.unsqueeze(0)

x = model.embedding(inputs).transpose(1, 2)
x = model.encoder.inference(x)

In [28]:
x = model.inference(inputs, speaker_id)

entered None


In [31]:
if 1: 
    print('oi')

oi


In [30]:
x

(tensor([[[ 0.2762,  0.3342, -0.2501,  ...,  0.0106, -0.0244, -0.2606],
          [-0.0091,  0.0212, -0.0278,  ...,  0.2481, -0.1136, -0.1236],
          [-0.1327,  0.0782, -0.2828,  ...,  0.3306,  0.0007,  0.1216],
          ...,
          [-0.1022, -0.0079,  0.0350,  ...,  0.0443, -0.0281, -0.1475],
          [-0.1101, -0.2710, -0.2979,  ...,  0.4224,  0.1402, -0.0630],
          [-0.0688,  0.0957, -0.0676,  ..., -0.0589, -0.2888, -0.0222]]],
        device='cuda:0'),
 tensor([[[ 2.7623e-01,  3.3417e-01, -2.5013e-01,  ...,  1.6349e+00,
           -2.4422e-02, -2.6056e-01],
          [-4.6211e+00,  2.1195e-02, -2.4959e-01,  ...,  2.4815e-01,
            2.8291e+00, -1.2356e-01],
          [-2.2278e+00,  5.0520e-01,  1.6054e+00,  ..., -8.8556e-01,
            6.8603e-04,  1.5414e+00],
          ...,
          [-1.0224e-01,  4.1707e+00,  3.7452e-02,  ...,  4.4334e-02,
           -2.8128e-02,  1.6176e+00],
          [-5.4382e-01, -9.3915e-01, -2.9789e-01,  ...,  4.2243e-01,
            1

In [23]:
speaker_ids = speaker_id
gst = True
num_speakers = 3
style_mel = None

if gst:
    # B x gst_dim
    inputs, encoder_outputs, logits = model.compute_gst(x,
                                       style_mel,
                                       speaker_embeddings if model.gst_use_speaker_embedding else None, style_agg = 'add')
# if num_speakers > 1:
#     if not model.embeddings_per_sample:
#         speaker_embeddings = model.speaker_embedding(speaker_ids)[:, None]
#     encoder_outputs = model._concat_speaker_embedding(encoder_outputs, speaker_embeddings)

entered None


In [24]:
inputs.shape

torch.Size([1, 120, 512])

In [20]:

encoder_outputs.shape

torch.Size([1, 1, 512])

In [21]:
rand = torch.rand(1, 1, 512)
model._add_speaker_embedding(encoder_outputs, rand).shape

torch.Size([1, 1, 512])

In [10]:
vetor_A = [1,1,1,1]
vetor_B = [0,1,4,1]

vetor_A = torch.Tensor(vetor_A)
vetor_B = torch.Tensor(vetor_B)

vetor_A = vetor_A.unsqueeze(0).unsqueeze(1)

vetor_B = vetor_B.unsqueeze(0)

print(vetor_A.shape, vetor_B.shape)

model._add_speaker_embedding(vetor_A, vetor_B),model._add_speaker_embedding(vetor_A, vetor_B).shape 

torch.Size([1, 1, 4]) torch.Size([1, 4])


(tensor([[[1., 2., 5., 2.]]]), torch.Size([1, 1, 4]))

In [11]:
import torch.nn.functional as F
F.normalize

<function torch.nn.functional.normalize(input, p=2, dim=1, eps=1e-12, out=None)>

In [12]:
F.normalize(vetor_A, dim = -1)

tensor([[[0.5000, 0.5000, 0.5000, 0.5000]]])

In [13]:
F.normalize(vetor_B, dim = -1)

tensor([[0.0000, 0.2357, 0.9428, 0.2357]])

In [14]:
! python TTS/bin/train_tts.py --config_path "./experiments/debug_add_emb/config.json" --experiment_folder "./experiments/debug_add_emb/" --continue_path ""

 > Using CUDA:  True
 > Number of GPUs:  1

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:
 | > hop_length:256
 | > win_length:1024
 | > Found 10 files in D:\Mestrado\Emotion Audio Synthesis (TTS)\repo_final\pt_etts\experiments\debug_linear_logits
Training with 2 speakers: marco, rosana
Training with 2 styles + neutral: happy, neutral
 > Using model: Tacotron2
Training with 2 speakers and 2 styles
Use style target = False
Use semi supervised = False

 > Model has 60796308 parameters

 > EPOCH: 0/3

 > Number of output frames: 7

 > Data

Traceback (most recent call last):
  File "TTS/bin/train_tts.py", line 888, in <module>
    main(args)
  File "TTS/bin/train_tts.py", line 756, in main
    global_step, epoch, amp, speaker_mapping, style_mapping)
  File "TTS/bin/train_tts.py", line 202, in train
    for num_iter, data in enumerate(data_loader):
  File "C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torch\utils\data\dataloader.py", line 352, in __iter__
    return self._get_iterator()
  File "C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torch\utils\data\dataloader.py", line 294, in _get_iterator
    return _MultiProcessingDataLoaderIter(self)
  File "C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torch\utils\data\dataloader.py", line 801, in __init__
    w.start()
  File "C:\Users\lucas\Anaconda3\envs\m_audio\lib\multiprocessing\process.py", line 105, in start
    self._popen = self._Popen(self)
  File "C:\Users\lucas\Anaconda3\envs\m_audio\lib\multiprocessing\context.py", line 223, in _Pope

In [16]:
text_cleaner = [CONFIG.text_cleaner]
text_cleaner

['phoneme_cleaners']

In [17]:
# s = '''Um professor, para se tornar doutor, investe, pelo menos, 8 anos de sua vida ( se partir da graduação direto pro doutorado). Se fizer graduação, mestrado e doutorado, são 10 anos. Se fizer graduação, especialização, mestrado e doutorado, lá se vão 12 anos. Sim, 12 anos de universidade. Todo esse tempo labutando com ensino, pesquisa e extensão. Produzindo, apresentando e publicando trabalhos.  Participando de debates e congressos. Dando aula. Ministrando cursos. Participando de bancas. Emitindo pareceres. Orientando artigos, projetos de conclusão de cursos, TCC's e etc, etc, etc. De repente, todo esse conhecimento arduamente produzido dentro dos parâmetros científicos e acadêmicos passa a não valer  nada ante a palavra de um esquizóide qualquer que define a forma como nós, professores, devemos conduzir o trabalho para o qual nos preparamos ao longo da vida e diuturnamente - pesquisando e estudando métodos e teorias, produzindo e submetendo nossas ideias à comunidade científica - que, todos sabemos, não é muito conhecida pela generosidade. Sem comentar os diversos perrengues -  familiares, socioemocionais, financeiros e de saúde -  que enfrentamos durante e  em virtude dessa caminhada extenuante e tão mal recompensada. Ainda assim, contra tudo e todos, persistimos cantando no mal tempo, porque temos a certeza daquilo que fazemos, e assentamos nossa autoridade e dignidade no conhecimento que produzimos. 
# Antes, a violência que se abatia sobre nós, profissionais da educação, era apenas financeira e a desvalorização social era decorrente dessa desvalorização financeira: '' Você é só uma professorazinha, ganha pouco''. 
# Agora, a violência é pior: a palavra e a autoridade intelectual do professor -  que ainda mantínhamos, apesar de tudo -  são jogados por terra. 
# Somos confrontados por pessoas que não leem, não têm conhecimento de absolutamente nada e que se informam apenas por youtubers, líderes religiosos delirantes ou figuras obscuras sem nenhum (re)conhecimento científico e que são  merecidamente proscritas do meio acadêmico.  Ainda por cima, somos chamados de '' doutrinadores'', acusados e vilipendiados das piores formas possíveis sem que os difamadores se retratem ou sofram as devidas punições por suas afirmações criminosas que têm como intuito deslegitimar, mais uma vez, a profissão docente nos violentando moralmente. E, como se não bastasse, há ''professores'' que se alinham a esse  evidente projeto de destruição da educação no país. Não me admira que a profissão esteja entre as menos procuradas pelos jovens, e que esteja, infelizmente, caminhando para a extinção.'''

import numpy as np

s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# s = "e é"

seq = np.asarray(text_to_sequence(s, text_cleaner), dtype=np.int32)
chars_var = torch.from_numpy(seq).unsqueeze(0).cuda()

# style_wav = {'1': 0.3} # Parece um token interessante

# f = 0.1
# style_wav = {}
# n_tokens = 3
# for i in range(n_tokens):
#     style_wav[str(i)] = f

style_wav = './data/LJSpeech/LJSpeech-1.1/wavs/LJ001-0003.wav'

speaker_embedding = None
speaker_id = 2


# style_wav = None

use_cuda = True

wav, alignment, decoder_output, postnet_output, stop_tokens, _, logits = synthesis(
    model.cuda(),
    s,
    CONFIG,
    use_cuda,
    ap,
    speaker_id=speaker_id,
    speaker_embedding=speaker_embedding,
    style_wav=style_wav,
    truncated=False,
    enable_eos_bos_chars=CONFIG.enable_eos_bos_chars, #pylint: disable=unused-argument
    use_griffin_lim=True,
    do_trim_silence=False)

In [18]:
wav.shape

(5120,)